Variable   Type      Data/Info
------------------------------
os         module    <module 'os' from '/home/<...>da3/lib/python3.8/os.py'>
sys        module    <module 'sys' (built-in)>


In [65]:
import numpy as np

def reg_model(beta, x):
    return beta[0]+beta[1]*x

def mse(beta, x, y):
    ypred = reg_model(beta,x)
    ssr = np.sum((ypred-y)**2)
    mean_squared_error = ssr/len(ypred)
    return mean_squared_error    

def grad_mse(beta, x, y):
    ypred = reg_model(beta,x)
    gradient = np.array([0,0])
    gradient[0] = 2/len(ypred)*np.sum(ypred-y)
    gradient[1] = 2/len(ypred)*np.sum((ypred-y)*x)    
    return gradient

def grad_descent(x, y, tol=1e-10, learn_rate=0.001, maxiter=400, thetaseed=[-10.0,20.0]):
    theta = thetaseed
    for iterate in range(maxiter):
        # Cost Function Val
        cost = mse(theta, x, y)
        gradient_vector = grad_mse(theta,x,y)
        theta[0]-=learn_rate*gradient_vector[0]
        theta[1]-=learn_rate*gradient_vector[1]        
        # Cost Function Revaluation
        cost_new = mse(theta,x,y)
        DeltaJ = abs(cost_new-cost)
        print(iterate," ",cost," (",theta[0],",",theta[1],")")
        if DeltaJ<tol:
            print("Convergence!")
            print(DeltaJ)
            break
    return theta